In [1]:
import pandas as pd
import numpy as np

In [192]:
def cpra_type(df):
    dfn = df.copy()
    dfn[['#CHROM', 'REF', 'ALT']] = dfn[['#CHROM', 'REF', 'ALT']].astype(str)
    dfn['POS'] = dfn['POS'].astype(int)
    return dfn

In [193]:
df = pd.read_excel('01_xlsx_files/vars_facts_together.xlsx')
gene_lst = pd.read_excel('01_xlsx_files/2021-08-06_gene list confluence.xlsx', sheet_name='Genes (A-Z)')
df = pd.merge(df, gene_lst[['Gene', 'Master_Erbgang Predisposition(s)']], left_on='HUGO_Symbol', right_on='Gene',
              how='left').drop(axis=1, columns='Gene')
df = cpra_type(df)

pat_dat_an = pd.read_excel(
    '01_xlsx_files/2021-10-11_NCT_MASTER_Onkostar_prioritzed_PIDs_2021-07-06_10-05-37_323.xlsx', 
    sheet_name='Onkostar_Anmeldung zum MASTER P')
pat_dat_tb = pd.read_excel(
    '01_xlsx_files/2021-10-11_NCT_MASTER_Onkostar_prioritzed_PIDs_2021-07-06_10-05-37_323.xlsx', 
    sheet_name='Onkostar_8_Tumorboard_2021-07-0')
all_vars = pd.read_pickle('00_dataframes/all_annotated_vars')
all_vars = cpra_type(all_vars)

In [194]:
df1 = df[['#CHROM', 'POS', 'REF', 'ALT', 'PosExonRefSeqAccession', 'PosExon_type', 
          'ClinVar_Pathogenicity', 'hgsv_new', 'INFO_IGV', 'too_low_coverage',
          'splice_change', 'shortcut', 'Splice Project Gene Priority', 'var_in_blood_perc',
          'var_in_tumor_perc', 'counts_coverage', 'perc_coverage', 'loh', 'SNP_nearby',
          'WGS_exome', 'NMD_seen', 'info_cDNA', 'more_consequences', 'shortcut_v', 
          'HUGO_Symbol', 'prob_effect', 'splice_site', 'variants_after_pos',
          'seen_in_gepado', 'other_variants_in_patient', 'Anmerkungen',
          'own_ACMG_assessment', 'own_ACMG_class', 'qci_disease', 'cancer_types', 
          'ID_Person', 'KGE_HBOC', 'patient_data_used', 'patient_data',
          'Master_Erbgang Predisposition(s)']].copy().drop_duplicates(subset=['#CHROM', 'POS']
                                                                     ).reset_index(drop=True)
dfm = pd.merge(all_vars[['#CHROM', 'POS', 'REF', 'ALT', 'PID_trans']][all_vars['POS'].isin(df1['POS'])], df1,
               on=['#CHROM', 'POS', 'REF', 'ALT'], how='inner')

In [197]:
df_pat = pd.merge(dfm, pat_dat_an, left_on='PID_trans', right_on='Patienten-ID', suffixes=('', '_an'))
df_pat = pd.merge(df_pat, pat_dat_tb, left_on='PID_trans', right_on='Patienten-ID', suffixes=('_an', '_tb'))

In [200]:
df_pat.to_excel('02_output_analysis/2021-10-11_onkostar_merged.xlsx', index=False)

### Tumor Board Slide Input

In [1]:
import pandas as pd
import numpy as np
from pptx import Presentation

In [12]:
prs = Presentation('02_output_analysis/VEAUMV_TuBoSlide.pptx')
text_runs = []

In [13]:
for slide in prs.slides:
    for shape in slide.shapes:
        if not shape.has_text_frame:
            continue
        for par in shape.text_frame.paragraphs:
            for run in par.runs:
                text_runs.append(run.text)

In [14]:
for i in text_runs:
    if 'Purity' in i:
        print(i.split('Purity: ')[1])

78%


In [15]:
text_runs

['VEAUMV – (BER-187) Kolonkarzinom, HEP Met',
 'snv = single nucleotide variant, in = insertion, del = deletion, amp = amplification, exp+ = increased expression',
 'exp- = decreased expression, fus = fusion, sv = structural variant',
 'Sequencing: WES+RNA       Ploidy: 2       Purity: 78%',
 'HRD: 2 LST: 2        MSIsensor: 0.33',
 'Signatures: AC18 unknown 79% CI=[76,106], AC10 altered POL E 17% CI=[14,27], AC4 tobacco mutagens, benzo(a)pyrene 5% CI=[-9,42]',
 'RNA',
 ' 81% mapping rate, 47% duplication rate, 85M library size',
 '0',
 '100',
 'Mutational Load of the Tumor',
 '                                                                                                         ▼']